In [1]:
# Importing Required Modules 
import cv2
import numpy as np
from rembg import remove 
from PIL import Image 

def enhance_and_crop_passport_image(input_image_path, output_image_path, size_mm=(35, 45), dpi=300):

    # Calculate passport size in pixels
    size_pixels = (int(size_mm[0] * dpi / 25.4), int(size_mm[1] * dpi / 25.4))
    
    # Read the input image
    img = cv2.imread(input_image_path)
    if img is None:
        print("Error: Unable to load the image.")
        return
    
    # Convert to grayscale for face detection
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Use a pre-trained Haar Cascade for face detection
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    if len(faces) == 0:
        print("No face detected in the image.")
        return
    
    # Select the largest detected face
    x, y, w, h = max(faces, key=lambda rect: rect[2] * rect[3])
    
    # Add padding around the face for a better crop
    padding = 0.3  # Add 30% padding
    x = max(0, int(x - padding * w))
    y = max(0, int(y - padding * h))
    w = int(w * (1 + 2 * padding))
    h = int(h * (1 + 2 * padding))
    
    # Crop the image around the face
    cropped_img = img[y:y+h, x:x+w]
    
    # Resize the image using Lanczos interpolation for high quality
    resized_img = cv2.resize(cropped_img, size_pixels, interpolation=cv2.INTER_LANCZOS4)
    
    # Enhance the image quality: apply sharpening
    kernel = np.array([[0, -1, 0],
                       [-1, 5, -1],
                       [0, -1, 0]])
    enhanced_img = cv2.filter2D(resized_img, -1, kernel)
    
    # Processing the image 
    input_image = enhanced_img 

    # Removing the background from the given Image 
    output_image = remove(input_image) 
    
    # Save the enhanced passport photo
    cv2.imwrite(output_image_path, output_image)
    print(f"Enhanced passport photo saved to {output_image_path}")

input_path = "C:/Users/Aryan/Desktop/edubricz/3.jpg"  
output_path = "C:/Users/Aryan/Desktop/edubricz/passport_image3.jpg"  
enhance_and_crop_passport_image(input_path, output_path)


Enhanced passport photo saved to C:/Users/Aryan/Desktop/edubricz/passport_image3.jpg
